In [ ]:
import pandas as pd
import wandb
import os
from datetime import datetime
from rich.progress import Progress

api = wandb.Api(timeout=60)
entity = '<REDACTED>'
project = 'MORL-Generalization'
if not entity:
    raise ValueError("Entity not provided and environment variable 'WANDB_ENTITY' is not set.")
print(f"Fetched entity from environment variable 'WANDB_ENTITY': {entity}.")

if not project:
    raise ValueError("Project not provided and environment variable 'WANDB_PROJECT' is not set.")
print(f"Fetched project from environment variable 'WANDB_PROJECT': {project}.")

In [ ]:
filters = {"group": "domain_randomization", "tags": {"$in": ["MOHopperDR-v5"]}}
try:
    runs_sample = api.runs(path=f"{entity}/{project}", per_page=1)
    total_runs = len(runs_sample)
except Exception as e:
    raise ValueError(f"Invalid entity '{entity}' or project '{project}': {str(e)}\n\nAlso, make sure you are properly authenticated. You can authenticate by using 'wandb.login() or setting the environment variable 'WANDB_API_KEY'")

In [ ]:
# Default CSV file name format
date_str = datetime.now().strftime("%m%d%y")
output_file = f"{entity}-{project}-{date_str}.csv"

all_runs_data = []
counter = 0
with Progress() as progress:
    task = progress.add_task("[cyan]Fetching runs...", total=total_runs)

    last_created_at = None
    while not progress.finished:
        filters = {"group": "domain_randomization"}
        if last_created_at:
            filters["created_at"] = {"$gt": last_created_at}

        runs = api.runs(path=f"{entity}/{project}", per_page=100, order="created_at", filters=filters)
        for run in runs:
            if run.state != "finished":
                continue
            run_data = {
                "name": run.name,
                "state": run.state,
                **run.summary._json_dict,
            }
            all_runs_data.append(run_data)
            progress.update(task, advance=1)
        if len(runs) > 0:
            last_created_at = runs[-1].created_at

df = pd.DataFrame(all_runs_data)
print(f"There are {len(df)} runs fetched.")

In [ ]:
df

In [ ]:
# Split the 'name' column into 'env_id', 'algorithm', 'seed', and 'time'
df[['env_id', 'name', 'seed', 'time']] = df['name'].str.split('__', expand=True)

# Drop rows with missing global_step
df = df.dropna(subset=['global_step'])

# Convert global_step to integer to ensure correct sorting
df['global_step'] = df['global_step'].astype(int)

# Filter for only columns that start with "eval/", "name", and "global_step"
columns_to_keep = df.filter(regex='^(eval/|name|global_step|env_id|seed)').columns
df = df[columns_to_keep]

# Remove the "eval/" prefix from the column names
df.columns = df.columns.str.replace('^eval/', '', regex=True)

# Create the directory structure and save CSV files for each seed
output_directory = 'data/wandb_summary'  # Base directory where everything will be saved

if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Loop through each unique combination of 'env_id' and 'algorithm'
for (env_id, algorithm), group in df.groupby(['env_id', 'name']):
    # Create folder for each env_id
    env_dir = os.path.join(output_directory, env_id)
    if not os.path.exists(env_dir):
        os.makedirs(env_dir)
    
    # Create folder for each algorithm within the corresponding env_id folder
    algo_dir = os.path.join(env_dir, algorithm)
    if not os.path.exists(algo_dir):
        os.makedirs(algo_dir)
    
    # Loop through each seed and save the corresponding data as a CSV file
    for seed, seed_data in group.groupby('seed'):
        # Sort by global_step in ascending order
        seed_data = seed_data.sort_values(by='global_step')

        # Set global_step as the index
        seed_data.set_index('global_step', inplace=True)
        
        # Ensure that rows with the same global_step are merged (if necessary)
        # This will collapse rows with the same 'global_step' by taking non-null values
        seed_data = seed_data.groupby('global_step').first()

        # Save the data for this seed
        seed_file_path = os.path.join(algo_dir, f'seed_{seed}.csv')
        seed_data.to_csv(seed_file_path, index=True)

print("Files have been successfully created.")
